In [17]:
production_df.select_dtypes(exclude="object")

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329,-5170.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469,-9118.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0,1
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458,-2175.0,...,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0,0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866,-2000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0,0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191,-4000.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,0,121500.0,412560.0,17473.5,270000.0,0.002042,-19970,-5169,-9094.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0
48740,456222,2,157500.0,622413.0,31909.5,495000.0,0.035792,-11186,-1149,-3015.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
48741,456223,1,202500.0,315000.0,33205.5,315000.0,0.026392,-15922,-3037,-2681.0,...,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0,0
48742,456224,0,225000.0,450000.0,25128.0,450000.0,0.018850,-13968,-2731,-1461.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,0


In [18]:
import pandas as pd

from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, DataQualityPreset

import joblib




loaded_model=joblib.load("./models/best_model.pkl")

reference_df=pd.read_csv("./data/application_train.csv")
production_df=pd.read_csv("./data/application_test.csv")
reference_df=reference_df.drop("SK_ID_CURR",axis=1)
y_true=reference_df["TARGET"]
#reference_df=reference_df.drop("TARGET",axis=1)

# numerical_features = ["AMT_INCOME_TOTAL"]
# column_mapping = ColumnMapping()

# column_mapping.datetime = "date"
# column_mapping.numerical_features = numerical_features


reference_pred=loaded_model.predict(reference_df)
production_pred=loaded_model.predict(production_df)

production_df["TARGET"]=loaded_model.predict(production_df)

report = Report(metrics=[
    DataDriftPreset(), 
])

report.show(mode='inline')


report.run(reference_data=reference_df, current_data=production_df,
           #column_mapping=column_mapping
          )
report

MemoryError: Unable to allocate 246. MiB for an array with shape (105, 307511) and data type float64

In [3]:
from sklearn.impute import SimpleImputer


def fill_cat(df, most_frequent=False):
    # Copie du DataFrame pour éviter toute modification de l'original
    df_tmp = df.copy()
    
    # Sélection des colonnes catégorielles dans le DataFrame
    # et stockage dans une liste
    cat_cols = [col for col in df.columns if df[col].dtype == 'object']

    # Remplissage des valeurs manquantes dans chaque colonne catégorielle
    # en utilisant la stratégie "most_frequent" de SimpleImputer
    if most_frequent: 
        imputer = SimpleImputer(strategy='most_frequent')
        # Application de la méthode fit_transform de SimpleImputer 
        # pour remplacer les valeurs manquantes par la valeur la plus fréquente
        df_tmp[cat_cols] = imputer.fit_transform(df_tmp[cat_cols])
    # Sinon, remplir les valeurs manquantes dans chaque colonne catégorielle 
    # avec la chaîne "Other"
    else:
        df_tmp[cat_cols] = df_tmp[cat_cols].fillna("Other")

    # Retourne les valeurs du DataFrame rempli
    return df_tmp.values


def fill_num(df, strategy="median"):
    """
    Remplit les valeurs manquantes dans les colonnes numériques d'un DataFrame avec une stratégie donnée.

    Args:
        df (pandas.DataFrame): DataFrame contenant les colonnes numériques à remplir.
        strategy (str): Stratégie à utiliser pour remplir les valeurs manquantes. Les options sont "mean",
            "median", "most_frequent" et "constant". La valeur par défaut est "median".

    Returns:
        numpy.ndarray: Un tableau NumPy contenant les colonnes numériques avec les valeurs manquantes remplies.

    """
    # Copier le DataFrame d'entrée pour éviter de modifier l'original
    df_tmp = df.copy()

    # Créer un objet SimpleImputer avec la stratégie spécifiée
    imputer = SimpleImputer(strategy=strategy)

    # Sélectionner les colonnes numériques dans le DataFrame
    cols = [col for col in df.columns if df[col].dtype == "float64"]

    # Remplacer les valeurs manquantes dans chaque colonne numérique
    df_tmp[cols] = imputer.fit_transform(df_tmp[cols])

    # Retourner les colonnes numériques avec les valeurs manquantes remplies sous forme de tableau NumPy
    return df_tmp.values